In [ ]:
#!/usr/bin/python3
# coding: utf-8
# MHLW

In [ ]:
from datetime import datetime as dt
from datetime import timedelta as td
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, show_and_save_plotly, \
    make_exp_fit_graph

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')

In [ ]:
dt_range = (dt.today() - dt.strptime(DT_OFFSET, "%Y/%m/%d")).days
from_date = dt.strptime(DT_OFFSET, "%Y/%m/%d")
xbins = [from_date + td(days=i) for i in range(dt_range)]
days2pred = 4 * 7
xbins_pred = [from_date + td(days=i) for i in range(dt_range + days2pred)]

In [ ]:
# 実効再生産数
ogiwara_uri = "https://raw.githubusercontent.com/kaz-ogiwara/covid19/master/data/"
ern_file = "effective_reproduction_number.csv"
download_if_needed(ogiwara_uri, ern_file)

In [ ]:
ern_data_arr = []
for i in np.arange(dt_range):
    ern_data_arr.append([i, 0, 0, 0])
csv2array(ern_data_arr, 'ern', ern_file, 1)
ern_data_np = np.array(ern_data_arr)
#print(ern_data_np[:,1])

In [ ]:
X = np.arange(0, len(ern_data_arr))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]
y_ern = get_gpr_predict(X, ern_data_np[:, 1], X_pred, 80, 10, 200)

In [ ]:
if False:
    fig, ax = create_basic_plot_figure()
    plt.xlim(xbins_pred[44], xbins_pred[-1])
    plt.ylim(0, max(ern_data_np[:, 1])+0.1)
    plt.ylabel('実効再生産数', fontname=FONT_NAME)
    plt.plot(xbins_pred, y_ern, '-.', linewidth=1, color='#ff8080')
    plt.bar(xbins, ern_data_np[:, 1], color='#00c000', edgecolor='#00c000', alpha=.5)
    plt.legend(labels=["予測値", "実効再生産数"], loc='upper left', borderaxespad=0, fontsize=12, prop={"family":FONT_NAME})
    plt.title('全国 新型コロナ予測 実効再生産数', fontname=FONT_NAME)
    show_and_clear(fig, "docs/images/ogiwara-ern.jpg")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=xbins, y=ern_data_np[:, 1], name="実効再生産数", opacity=0.5))
fig.add_trace(go.Scatter(x=xbins_pred, y=y_ern, mode='lines', name='予測値', line=dict(width=1)))
fig.update_layout(
    xaxis=dict(title='日付', type='date', dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins[44], xbins_pred[-1]]),
    yaxis=dict(title='実効再生産'),
    title='全国 新型コロナ 実効再生産数 ({})'.format(today_str),
)
show_and_save_plotly(fig, "ogiwara-ern.jpg", js=False)

In [ ]:
tw_body_ern = "全国 新型コロナ 実効再生産数 ({})".format(today_str)
tw_body_ern += " https://geneasyura.github.io/cov19-hm/tokyo.html "
tw = get_twitter()
tweet_with_image(tw, "docs/images/ogiwara-ern.jpg", tw_body_ern)

In [ ]:
title = '全国 新型コロナ 新規陽性者移動平均/累乗近似 (' + today_str + ')'
xdata = np.array(xbins)
#ydata = all_data_np[:, offsets['posis']]
ydata = posis_mov_mean
xos = 265
make_exp_fit_graph(tw,
    xdata[xos:], ydata[xos:],
    title, "mhlw-fit.jpg",
    "mhlw-doubling-time.html", "mhlw.html")